<a href="https://colab.research.google.com/github/voodoohop/colabasaservice/blob/master/colabs/deep-daze.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text to Image
### Method: CLIP + Siren
This model combines an image classifier called **CLIP** from *OpenAI* and an image generator called **SIREN.**

The process works like this:
1.  The **SIREN** model at first generates a completely random image
2.  The **CLIP** image classifier looks at the image and compares it with the given text caption
3.  Using this feedback the weights of the **SIREN** model are adjusted slightly to better approximate the image caption
4.  These steps are repeated until we stop it manually
---
In order to run click the [Colab] link. And then select *Runtime -> Run all* from the menu bar.

[colab]: https://colab.research.google.com/github/voodoohop/colabasaservice/blob/master/colabs/deep-daze.ipynb

# Parameters

In [107]:
text = 'An illustration of a black and white cat'  #@param {type: "string"}
args = '--lower_bound_cutout 0.5 --image-width=384 --iterations=1000 --epochs=2 --save-every=2 --save-progress=True --open_folder=False --overwrite=True --save_video=True --seed=123'  #@param {type: "string"}

parameters = {
    "text": text,
    "args": args
}

## Code (IPFS Connection)

In [ ]:
!npm install -g npm@latest
!npm install -g --force --unsafe-perms node-pre-gyp bcrypt ipfs
!pip install ipfshttpclient

In [1]:
IPFS_PEER="18.157.173.110"
!nohup jsipfs daemon --enable-namesys-pubsub &
!sleep 5
#!jsipfs config Addresses.API /ip4/127.0.0.1/tcp/5002
#!jsipfs config --json API.HTTPHeaders.Access-Control-Allow-Origin '["*"]'
#!jsipfs config --json API.HTTPHeaders.Access-Control-Allow-Methods '["PUT", "GET", "POST"]'
!jsipfs swarm connect /ip4/$IPFS_PEER/tcp/4002/p2p/QmWxrFJqurnyedzJ5rzpgycTqSikJ7kcy7eHmqYnndKhwr

nohup: appending output to 'nohup.out'
connect /ip4/18.157.173.110/tcp/4002/p2p/QmWxrFJqurnyedzJ5rzpgycTqSikJ7kcy7eHmqYnndKhwr success


In [ ]:
!pip install ipfshttpclient
#!pip install git+https://github.com/voodoohop/py-ipfs-http-client.git

In [113]:
import IPython
from google.colab import output

display(IPython.display.Javascript('''
  const iframe = document.createElement("iframe")
  iframe.src = "http://localhost:3000/colabasaservice/incolab.html"
  const channel = new BroadcastChannel("inColab")
  channel.onmessage = ({data:link}) => {
    console.log("passing on link from colab",link)
    iframe.contentWindow.postMessage(link,"*");
  }
  function handleEvent(e) {
      console.log(e.detail) // outputs: {foo: 'bar'}
  }

  window.document.addEventListener('myCustomEvent', handleEvent, false)
  
  document.querySelector("#output-area").appendChild(iframe)
'''))

def update_state(link):
  output.eval_js('new BroadcastChannel("inColab").postMessage("'+link+'")');

#update_state("heeeey")

<IPython.core.display.Javascript object>

In [16]:
#!pip install aioipfs
import sys
import asyncio

import aioipfs

client = aioipfs.AsyncIPFS(host="localhost",port=5002)
async def add_folder(path):
    print("addd")
    print("simple add", await client.core.add_str('AaasdsdasdBCD'))
    #print("complex add", await client.core.add(path,recursive=True))
    async for added_file in client.core.add_str(path):
        print('Imported file {0}, CID: {1}'.format(
            added_file['Name'], added_file['Hash']))

    

#loop = asyncio.get_event_loop()
asyncio.ensure_future(add_folder("/content/ipfs_root"))



Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7fb1b7dc9050>


<Task pending coro=<add_folder() running at <ipython-input-16-02002f3891e9>:8>>

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: coroutine 'CoreAPI.add_str' was never awaited
  if sys.path[0] == '':


addd
simple add {'Name': 'QmZnvAege7RFgeBPffmHDGWWzd8AA8tqqL59MKCYrhB5fY', 'Hash': 'QmZnvAege7RFgeBPffmHDGWWzd8AA8tqqL59MKCYrhB5fY', 'Size': 21, 'Mode': '0644'}


In [153]:
import ipfshttpclient #{IPFS_PEER}
client = ipfshttpclient.connect(f"/dns/localhost/tcp/5002/http") 
ipfs_pinner = ipfshttpclient.connect(f"/ip4/{IPFS_PEER}/tcp/5002/http") 

ipfs_root = "/content/ipfs_root"

!rm -r $ipfs_root
!mkdir -p $ipfs_root

def add_and_publish(pin=False, root=ipfs_root):
  add_res = client.add(root, recursive=True)
  print("add_res",add_res[-1])
  ipfs_hash = add_res[-1]["Hash"]
  update_state(ipfs_hash)
  return "/ipfs/"+ipfs_hash

  #ipns_res = client.name.publish("/ipfs/"+ipfs_hash)
  #ipns_link = f"/ipns/{ipns_res['Name']}"
  if pin:
    print("pinres",ipfs_pinner.pin.add(ipfs_hash))
  #return ipns_link # f"/ipfs/{ipfs_hash}"

ipns_link = add_and_publish(pin=True)

print("IPNS_LINK",ipns_link)


add_res <ipfshttpclient.client.base.ResponseBase: {'Name': 'ipfs_root', 'Hash': 'QmUNLLsPACCz1vLxQVkXqqLX5R1X345qqfHbsf67hvA3Nn', 'Size': 4, 'Mode': '0755'}>
IPNS_LINK /ipfs/QmUNLLsPACCz1vLxQVkXqqLX5R1X345qqfHbsf67hvA3Nn


## CODE (DEEP-DAZE)

In [109]:
!pip install deep-daze --upgrade
!pip install stomp.py imageio-ffmpeg asyncio websockets varname --upgrade

Requirement already up-to-date: deep-daze in /usr/local/lib/python3.7/dist-packages (0.10.2)
Requirement already up-to-date: stomp.py in /usr/local/lib/python3.7/dist-packages (7.0.0)
Requirement already up-to-date: imageio-ffmpeg in /usr/local/lib/python3.7/dist-packages (0.4.3)
Requirement already up-to-date: asyncio in /usr/local/lib/python3.7/dist-packages (3.4.3)
Requirement already up-to-date: websockets in /usr/local/lib/python3.7/dist-packages (8.1)
Requirement already up-to-date: varname in /usr/local/lib/python3.7/dist-packages (0.6.3)


In [ ]:
from subprocess import Popen, PIPE, STDOUT
from sys import stdout
from glob import glob
import os
import stomp
import json
import itertools
from base64 import b64encode
from time import sleep
import requests
import os
from mimetypes import guess_type 
import re
flatten = itertools.chain.from_iterable

TEXT="A black and white technical illustration of merging a cat and a human through biohacking."
DEFAULT_PARAMS="--lower_bound_cutout 0.5 --image-width=384 --iterations=1000 --epochs=2 --save-every=2 --save-progress=True --open_folder=False --overwrite=True --save_video=True --seed=1234"
GITHUB_REPO="voodoohop/colabasaservice/blob/master/colabs/deep-daze.ipynb"
GITHUB_REPO_CLEAN = re.sub('[^0-9a-zA-Z]+', '_', GITHUB_REPO)
print(GITHUB_REPO_CLEAN)

def run(text, params=DEFAULT_PARAMS, watch_media_paths=["./*.jpg","./*.mp4"]):
  print("Running",text)
  for to_remove in watch_media_paths:
    for filepath in glob(to_remove):
      print("Deleting",filepath,"before starting.")
      os.remove(filepath)

  cmd = f'imagine {params} "{text}"'
  
  p = Popen(cmd, stdout = PIPE, 
          stderr = STDOUT, shell = True)

  found_media_files=[]
  log_out = open(f"{ipfs_root}/colabConsole.txt","a")
  while True:
    line = p.stdout.readline()#
    if not line: break

    stdout.write(line)
    log_out.write(line.decode("utf-8"))
    log_out.flush()
    #headers = { 
    #    "params": params,
    #    "text": text
    #}
    #conn.send(body=line.decode("utf-8"), headers=headers, destination='/topic/colabOut')
    
    new_found_media_files = get_matching_files(watch_media_paths)
    new_found_media_files.sort(key=os.path.getmtime)
    added_media_files = list(set(new_found_media_files)-set(found_media_files))
    found_media_files = new_found_media_files
    pin=False
    if len(added_media_files) > 0:
      for added in added_media_files:
        sleep(0.1)
        if len(added) > 0:
          !cp $added $ipfs_root
          print("Adding:", added,"to ipfs root")
          pin=True
    add_and_publish(pin)          

def get_matching_files(search_paths):
  return list(flatten([glob(search_path) for search_path in search_paths]))

sub = client.pubsub.subscribe("asd")
#conn.set_listener('', MyListener(conn,run))
print(sub)
#run(TEXT)
#conn.disconnect()
sleep(1000000)


In [ ]:
conn.disconnect()